In [1]:
from bs4 import BeautifulSoup as b
import requests as rqt
import pandas as pd
import time

In [2]:
URL = "https://www.pro-football-reference.com/draft/2000-combine.htm"

r = rqt.get(URL)

if r.status_code == 429:
    retry_after = r.headers.get('Retry-After')
    if retry_after:
        wait_time = int(retry_after)
        print(f"Rate limit exceeded. Retrying after {wait_time} seconds.")
        time.sleep(wait_time)

In [4]:
soup = b(r.text, "lxml")

table_head = soup.find("thead")
headers = table_head.find_all("th")

column_names = []
for i in headers:
    value = i.text
    column_names.append(value)

print(column_names)

['Player', 'Pos', 'School', 'College', 'Ht', 'Wt', '40yd', 'Vertical', 'Bench', 'Broad Jump', '3Cone', 'Shuttle', 'Drafted (tm/rnd/yr)']


In [5]:
df = pd.DataFrame(columns=['Player','DOB','Pos', 'stats_link', 'Ht', 'Wt', '40yd', 'Vertical', 'Bench', 'Broad Jump', '3Cone', 'Shuttle', 'Drafted (tm/rnd/yr)'])

In [6]:
table_body = soup.find("tbody")
body = table_body.find_all("tr")
Player_details = []

for i in body:
    th = i.find("th")

    if th:
        a = th.find("a")
        if a:
            name = a.text
            if name == 'Player':
                continue
            
            name_link = a["href"]
            if name_link:
                print("https://www.pro-football-reference.com" + str(name_link))
                P_Req = rqt.get("https://www.pro-football-reference.com" + str(name_link))
                if P_Req.status_code == 429:
                    retry_after = P_Req.headers.get('Retry-After')
                    if retry_after:
                        wait_time = int(retry_after)
                        print(f"Rate limit exceeded. Retrying after {wait_time} seconds.")
                        time.sleep(wait_time)
                P_soup = b(P_Req.text, "lxml")
                DOB_ele = P_soup.find(id="necro-birth")
                print(DOB_ele)
                if DOB_ele.text:
                    DOB = DOB_ele.text.split(',')[1][1 : 5]
                    print(DOB)
                else:
                    DOB = " "
                    print(DOB)


            else:
                name_link = "https://www.pro-football-reference.com/players/" + name[name.find(" ")+1] + "/" + name[name.find(" ")+1 : name.find(" ")+5] + name[0:2] + "00.htm"
                print(name_link)
                P_Req = rqt.get(name_link)
                if P_Req.status_code == 429:
                    retry_after = P_Req.headers.get('Retry-After')
                    if retry_after:
                        wait_time = int(retry_after)
                        print(f"Rate limit exceeded. Retrying after {wait_time} seconds.")
                        time.sleep(wait_time)
                P_soup = b(P_Req.text, "lxml")
                print(P_soup)
                DOB_ele = P_soup.find(id="necro-birth")
                print(DOB_ele)
                if DOB_ele.text:
                    DOB = DOB_ele.text.split(',')[1][1 : 5]
                else:
                    DOB = " "

                                

    td = i.find_all("td")
    for i in td:
        if i["data-stat"] == "pos":
            pos = i.text

        if i["data-stat"] == "college":
            stats = i.find("a")
            if stats:
                stats_link = stats["href"]
            else:
                stats_link = " "

        if i["data-stat"] == "height":
            height = i.text

        if i["data-stat"] == "weight":
            weight = i.text

        if i["data-stat"] == "forty_yd":
            forty_yd = i.text

        if i["data-stat"] == "vertical":
            vertical = i.text

        if i["data-stat"] == "bench_reps":
            bench_reps = i.text

        if i["data-stat"] == "broad_jump":
            broad_jump = i.text
        
        if i["data-stat"] == "cone":
            cone = i.text

        if i["data-stat"] == "shuttle":
            shuttle = i.text

        if i["data-stat"] == "draft_info":
            draft_info = i.text

    row = {'Player': name,'DOB': DOB, 'Pos': pos, 'stats_link': stats_link, 'Ht': height, 'Wt': weight, '40yd': forty_yd, 'Vertical': vertical, 'Bench': bench_reps, 'Broad Jump': broad_jump, '3Cone': cone, 'Shuttle': shuttle, 'Drafted (tm/rnd/yr)': draft_info}

    Player_details.append(row)

df = pd.DataFrame(Player_details)




https://www.pro-football-reference.com/players/A/AbraJo00.htm
<span data-birth="1978-05-06" id="necro-birth">
<a href="/friv/birthdays.cgi?month=5&amp;day=6">May 6</a>, 1978
    </span>
1978
https://www.pro-football-reference.com/players/A/AlexSh00.htm
<span data-birth="1977-08-30" id="necro-birth">
<a href="/friv/birthdays.cgi?month=8&amp;day=30">August 30</a>, 1977
    </span>
1977
https://www.pro-football-reference.com/players/A/AlfoDa20.htm
<span data-birth="1977-06-11" id="necro-birth">
<a href="/friv/birthdays.cgi?month=6&amp;day=11">June 11</a>, 1977
    </span>
1977
https://www.pro-football-reference.com/players/A/AndeRa21.htm
<span data-birth="1977-06-14" id="necro-birth">
<a href="/friv/birthdays.cgi?month=6&amp;day=14">June 14</a>, 1977
    </span>
1977
https://www.pro-football-reference.com/players/A/arianjak01.htm
<span data-birth="1978-01-26" id="necro-birth">
<a href="/friv/birthdays.cgi?month=1&amp;day=26">January 26</a>, 1978
    </span>
1978
https://www.pro-football-r

AttributeError: 'NoneType' object has no attribute 'text'

In [6]:
print(df)

Empty DataFrame
Columns: [Player, DOB, Pos, stats_link, Ht, Wt, 40yd, Vertical, Bench, Broad Jump, 3Cone, Shuttle, Drafted (tm/rnd/yr)]
Index: []


In [140]:
df.to_csv('D:\602\Project\Data\Data_2000.csv', index=False)